# Playing around to get an idea bout pos-list vs. seq performance

In [11]:
import numpy as np
import os
import pandas as pd

In [34]:
base_directory = "/Users/martin/Programming/compression_selection_v3/hyrise_calibration/calibration/2021-05-25T100031/results/"

def estimate_aggregates(csv_path):
    df = pd.read_csv(csv_path)
    
    seq_runtime = 0.0
    nonseq_runtime = 0.0
    
    temp_table_marker = np.where(df.TABLE_NAME.isnull(), 1.0, 0.0)
    non_grouping_marker = np.where(df.GROUP_BY_COLUMN_COUNT == 0, 1.0, 0.0)
    
    # If we don't group, the aggregation is sequential (if on non-shuffled table)
    df['non_grouping_runtime'] = (1 - temp_table_marker) * non_grouping_marker * ((1 - df.INPUT_SHUFFLEDNESS) * (df.GROUP_BY_KEY_PARTITIONING_NS + df.AGGREGATING_NS))
    
    # Grouping is sequential on unshuffled tables, otherwise not
    df['grouping_seq_runtime'] = (1 - temp_table_marker) * (1 - non_grouping_marker) * ((1 - df.INPUT_SHUFFLEDNESS) * df.GROUP_BY_KEY_PARTITIONING_NS)
    df['grouping_nonseq_runtime'] = (1 - temp_table_marker) * (1 - non_grouping_marker) * (df.INPUT_SHUFFLEDNESS * df.GROUP_BY_KEY_PARTITIONING_NS)
    
    df['aggregating'] = (1 - temp_table_marker) * (1 - non_grouping_marker) * df.AGGREGATING_NS
    
    projection = ["TABLE_NAME", "INPUT_SHUFFLEDNESS", "GROUP_BY_KEY_PARTITIONING_NS", "AGGREGATING_NS"]
    projection.extend(df.columns[-4:])
    return df[projection]

def estimate_joins(csv_path):
    df = pd.read_csv(csv_path)
    
    seq_runtime = 0.0
    nonseq_runtime = 0.0
    
    left_temp_table_marker = np.where(df.LEFT_TABLE_NAME.isnull(), 1.0, 0.0)
    right_temp_table_marker = np.where(df.RIGHT_TABLE_NAME.isnull(), 1.0, 0.0)
    
    # we only need to care about the materialization phase, nothing else
    , 'PROBE_SIDE_MATERIALIZING_NS'
    
    df['build_seq_runtime'] = ((1 - df.LEFT_INPUT_SHUFFLEDNESS) * (1 - df.IS_FLIPPED) * df.BUILD_SIDE_MATERIALIZING_NS + \
                               (1 - df.RIGHT_INPUT_SHUFFLEDNESS) * (df.IS_FLIPPED) * df.BUILD_SIDE_MATERIALIZING_NS) * \
                              (1 - left_temp_table_marker)
    df['build_nonseq_runtime'] = (df.LEFT_INPUT_SHUFFLEDNESS * (1 - df.IS_FLIPPED) * df.BUILD_SIDE_MATERIALIZING_NS + \
                                  df.RIGHT_INPUT_SHUFFLEDNESS * (df.IS_FLIPPED) * df.BUILD_SIDE_MATERIALIZING_NS) * \
                                 (1 - left_temp_table_marker)
    df['probe_seq_runtime'] = ((1 - df.LEFT_INPUT_SHUFFLEDNESS) * (1 - df.IS_FLIPPED) * df.PROBE_SIDE_MATERIALIZING_NS + \
                               (1 - df.RIGHT_INPUT_SHUFFLEDNESS) * (df.IS_FLIPPED) * df.PROBE_SIDE_MATERIALIZING_NS) * \
                              (1 - right_temp_table_marker)
    df['probe_nonseq_runtime'] = (df.LEFT_INPUT_SHUFFLEDNESS * (1 - df.IS_FLIPPED) * df.PROBE_SIDE_MATERIALIZING_NS + \
                                  df.RIGHT_INPUT_SHUFFLEDNESS * (df.IS_FLIPPED) * df.PROBE_SIDE_MATERIALIZING_NS) * \
                                 (1 - right_temp_table_marker)

    projection = ["LEFT_TABLE_NAME", "RIGHT_TABLE_NAME", "IS_FLIPPED", "LEFT_INPUT_SHUFFLEDNESS", "RIGHT_INPUT_SHUFFLEDNESS", "BUILD_SIDE_MATERIALIZING_NS", "PROBE_SIDE_MATERIALIZING_NS"]
    projection.extend(df.columns[-4:])
    return df[projection]


for workload in ["TPCH", "TPCDS", "JOB"]:
    directory = os.path.join(base_directory, workload, "dictionary_fsba")
    
#     aggregates = estimate_aggregates(os.path.join(directory, "aggregates.csv"))
#     display(aggregates)
#     display(aggregates.agg(['sum']))
    
    joins = estimate_joins(os.path.join(directory, "joins.csv"))
    display(joins)
    display(joins.agg(['sum']))


,LEFT_TABLE_NAME,RIGHT_TABLE_NAME,IS_FLIPPED,LEFT_INPUT_SHUFFLEDNESS,RIGHT_INPUT_SHUFFLEDNESS,BUILD_SIDE_MATERIALIZING_NS,PROBE_SIDE_MATERIALIZING_NS,build_seq_runtime,build_nonseq_runtime,probe_seq_runtime,probe_nonseq_runtime
0,partsupp,NaN,1,0.50,0.50,200061,533184,1.000305e+05,100030.50,0.0,0.0
1,partsupp,part,0,0.50,0.50,722891,862974,3.614455e+05,361445.50,431487.0,431487.0
2,partsupp,supplier,0,0.75,0.75,506623,188694,1.266558e+05,379967.25,47173.5,141520.5
3,partsupp,supplier,1,0.50,0.50,202586,6970973,1.012930e+05,101293.00,3485486.5,3485486.5
4,partsupp,part,1,0.00,0.00,1362854,16304692,1.362854e+06,0.00,16304692.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
764,lineitem,supplier,1,0.00,0.00,300575,104184331,3.005750e+05,0.00,104184331.0,0.0
765,lineitem,supplier,1,0.00,0.00,256832,286482502,2.568320e+05,0.00,286482502.0,0.0
766,supplier,nation,1,0.00,0.00,91881,324073,9.188100e+04,0.00,324073.0,0.0
767,lineitem,lineitem,1,0.50,0.50,75283043,496031002,3.764152e+07,37641521.50,248015501.0,248015501.0


,IS_FLIPPED,LEFT_INPUT_SHUFFLEDNESS,RIGHT_INPUT_SHUFFLEDNESS,BUILD_SIDE_MATERIALIZING_NS,PROBE_SIDE_MATERIALIZING_NS,build_seq_runtime,build_nonseq_runtime,probe_seq_runtime,probe_nonseq_runtime
sum,675,174.375,174.375,23613506890,86331014108,1.716547e+10,6.447026e+09,6.756697e+10,1.875147e+10


,LEFT_TABLE_NAME,RIGHT_TABLE_NAME,IS_FLIPPED,LEFT_INPUT_SHUFFLEDNESS,RIGHT_INPUT_SHUFFLEDNESS,BUILD_SIDE_MATERIALIZING_NS,PROBE_SIDE_MATERIALIZING_NS,build_seq_runtime,build_nonseq_runtime,probe_seq_runtime,probe_nonseq_runtime
0,store_returns,customer,0,0.0,0.0,425188.0,2344255.0,425188.0,0.0,2344255.0,0.0
1,store_returns,store_returns,1,0.0,0.0,45148.0,1706912.0,45148.0,0.0,1706912.0,0.0
2,store_returns,store,1,0.0,0.0,121439.0,3634996.0,121439.0,0.0,3634996.0,0.0
3,store_returns,date_dim,1,0.0,0.0,149530.0,23941981.0,149530.0,0.0,23941981.0,0.0
4,customer,store_returns,1,0.0,0.0,1372456.0,3145897.0,1372456.0,0.0,3145897.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2987,store_sales,catalog_sales,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2988,catalog_sales,warehouse,1,0.5,0.5,84514.0,20861893.0,42257.0,42257.0,10430946.5,10430946.5
2989,catalog_sales,call_center,1,0.5,0.5,87036.0,21884544.0,43518.0,43518.0,10942272.0,10942272.0
2990,catalog_sales,ship_mode,1,0.0,0.0,92886.0,23310807.0,92886.0,0.0,23310807.0,0.0


,LEFT_TABLE_NAME,RIGHT_TABLE_NAME,IS_FLIPPED,LEFT_INPUT_SHUFFLEDNESS,RIGHT_INPUT_SHUFFLEDNESS,BUILD_SIDE_MATERIALIZING_NS,PROBE_SIDE_MATERIALIZING_NS,build_seq_runtime,build_nonseq_runtime,probe_seq_runtime,probe_nonseq_runtime
sum,store_returnsstore_returnsstore_returnsstore_r...,customerstore_returnsstoredate_dimstore_return...,2618,781.0,781.0,7.142324e+09,1.602921e+11,5.061723e+09,2.080601e+09,1.443560e+11,1.593607e+10


,LEFT_TABLE_NAME,RIGHT_TABLE_NAME,IS_FLIPPED,LEFT_INPUT_SHUFFLEDNESS,RIGHT_INPUT_SHUFFLEDNESS,BUILD_SIDE_MATERIALIZING_NS,PROBE_SIDE_MATERIALIZING_NS,build_seq_runtime,build_nonseq_runtime,probe_seq_runtime,probe_nonseq_runtime
0,movie_companies,company_type,1,0.50,0.50,28793,78250,14396.5,14396.5,39125.00,39125.00
1,title,movie_companies,0,0.75,0.75,53784,100700,13446.0,40338.0,25175.00,75525.00
2,title,movie_companies,1,0.50,0.50,116590,19700463,58295.0,58295.0,9850231.50,9850231.50
3,cast_info,role_type,1,0.50,0.50,40469,176663,20234.5,20234.5,88331.50,88331.50
4,char_name,cast_info,0,0.50,0.50,139444,246494,69722.0,69722.0,123247.00,123247.00
...,...,...,...,...,...,...,...,...,...,...,...
5026,company_name,movie_companies,1,0.50,0.50,38095402,4740452,19047701.0,19047701.0,2370226.00,2370226.00
5027,movie_companies,cast_info,0,0.75,0.75,10211654,15397473,2552913.5,7658740.5,3849368.25,11548104.75
5028,movie_companies,cast_info,1,0.50,0.50,17714975,17799012,8857487.5,8857487.5,8899506.00,8899506.00
5029,name,cast_info,0,0.75,0.75,2710880,10963392,677720.0,2033160.0,2740848.00,8222544.00


,LEFT_TABLE_NAME,RIGHT_TABLE_NAME,IS_FLIPPED,LEFT_INPUT_SHUFFLEDNESS,RIGHT_INPUT_SHUFFLEDNESS,BUILD_SIDE_MATERIALIZING_NS,PROBE_SIDE_MATERIALIZING_NS,build_seq_runtime,build_nonseq_runtime,probe_seq_runtime,probe_nonseq_runtime
sum,movie_companiestitletitlecast_infochar_namecha...,company_typemovie_companiesmovie_companiesrole...,3426,2483.0625,2483.0625,26898075307,112571783821,1.253706e+10,1.436102e+10,5.740984e+10,5.516194e+10
